In [1]:

import os
import time
import random
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp

c:\Users\chama\torch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# ----------------------------
# Configuration 
# ----------------------------
BASE_DIR = os.path.abspath('.')  # working directory where this script/ipynb lives
DATA_DIR = os.path.join(BASE_DIR, 'data', 'Train')   # e.g. ./data/Train
CSV_PATH = os.path.join(BASE_DIR, 'Train.csv')       # e.g. ./Train.csv
TEST_DATA_DIR = os.path.join(BASE_DIR, 'data', 'Test')
GROUND_TRUTH_PATH = os.path.join(BASE_DIR, 'test_solution.csv')
SUBMISSION_PATH = os.path.join(BASE_DIR, 'submission.csv')

TEST2_DATA_DIR = os.path.join(BASE_DIR, 'data', 'Test2')
TEST2_SUBMISSION_PATH = os.path.join(BASE_DIR, 'submission_test2.csv')


IMAGE_HEIGHT, IMAGE_WIDTH = 256, 256
BATCH_SIZE = 8
EPOCHS = 80                #  run for 80 epochs
LR = 1e-4
ENCODER = 'efficientnet-b3'
ENCODER_WEIGHTS = 'imagenet'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_SAVE = os.path.join(BASE_DIR, 'unet_effnet_nuclei.pth')
CHECKPOINT_EVERY = 10
NUM_WORKERS = 0
THRESHOLD = 0.5

# Visualization settings: save visualizations every VIS_EPOCH_FREQ epochs
VIS_EPOCH_FREQ = 5             # save visual grid at epoch 5,10,15,...
VIS_SAMPLES = 4                # number of example images to show per grid
VIS_DIR = os.path.join(BASE_DIR, 'training_visuals')

os.makedirs(VIS_DIR, exist_ok=True)

print(f"Device: {DEVICE}")
print(f"Base dir: {BASE_DIR}")
print(f"Data dir: {DATA_DIR}")
print(f"Train CSV: {CSV_PATH}")
print(f"Test dir: {TEST_DATA_DIR}")
print(f"Test2 dir: {TEST2_DATA_DIR}")
print(f"Visualizations will be saved to: {VIS_DIR}")
print()


Device: cuda
Base dir: c:\Users\chama\Desktop\Project
Data dir: c:\Users\chama\Desktop\Project\data\Train
Train CSV: c:\Users\chama\Desktop\Project\Train.csv
Test dir: c:\Users\chama\Desktop\Project\data\Test
Test2 dir: c:\Users\chama\Desktop\Project\data\Test2
Visualizations will be saved to: c:\Users\chama\Desktop\Project\training_visuals



In [3]:

# ----------------------------
# Helper: RLE encode/decode
# ----------------------------
def rle_decode(mask_rle, shape):
    if pd.isna(mask_rle) or mask_rle == '':
        return np.zeros(shape, dtype=np.uint8)
    s = mask_rle.split()
    if not s:
        return np.zeros(shape, dtype=np.uint8)
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')


def rle_encode(mask):
    # mask: 2D numpy array, 1 - mask, 0 - background
    pixels = mask.T.flatten()  # to Fortran order
    # pad with zero at both ends to catch runs that start at index 0 or end at last
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] = runs[1::2] - runs[::2]
    # runs are start,length alternating
    if runs.size == 0:
        return ''
    return ' '.join(str(x) for x in runs)




In [4]:
# ----------------------------
# Dataset
# ----------------------------
class NucleiDataset(Dataset):
    def __init__(self, base_dir, mask_df=None, image_ids=None, transform=None, is_test=False):
        self.base_dir = base_dir
        self.mask_df = mask_df
        if image_ids is not None:
            self.ids = image_ids
        elif mask_df is not None:
            self.ids = list(mask_df['ImageId'].unique())
        else:
            # fallback: list directories in base_dir
            self.ids = [os.path.basename(p) for p in glob(os.path.join(base_dir, '*'))]
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        image_id = self.ids[idx]
        image_glob = glob(os.path.join(self.base_dir, image_id, 'images', '*'))
        if not image_glob:
            raise FileNotFoundError(f"Image for ID {image_id} not found in {self.base_dir}/{image_id}/images/")
        image_path = image_glob[0]
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape

        if self.is_test:
            mask = np.zeros((h, w), dtype=np.uint8)
        else:
            # build combined mask from all RLEs for the image id
            mask = np.zeros((h, w), dtype=np.uint8)
            records = self.mask_df[self.mask_df['ImageId'] == image_id]
            for _, r in records.iterrows():
                mask += rle_decode(r['EncodedPixels'], (h, w)).astype(np.uint8)
            mask = np.clip(mask, 0, 1)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask'].unsqueeze(0).float()
        else:
            # fallback: convert to tensor-like numpy arrays
            image = image.astype(np.float32) / 255.0
            image = np.transpose(image, (2,0,1))
            mask = np.expand_dims(mask, 0).astype(np.float32)

        return image, mask, image_id


In [5]:
# ----------------------------
# Transforms
# ----------------------------
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.3, border_mode=cv2.BORDER_CONSTANT),
    A.RandomBrightnessContrast(p=0.3),
    A.Resize(IMAGE_HEIGHT, IMAGE_WIDTH),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225), max_pixel_value=255.0),
    ToTensorV2(),
])

valid_transform = A.Compose([
    A.Resize(IMAGE_HEIGHT, IMAGE_WIDTH),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225), max_pixel_value=255.0),
    ToTensorV2(),
])



c:\Users\chama\torch_env\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [6]:
# ----------------------------
# Load dataset
# ----------------------------
if not os.path.exists(DATA_DIR) or not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"DATA_DIR or CSV_PATH not found. Expected {DATA_DIR} and {CSV_PATH}")

mask_df = pd.read_csv(CSV_PATH)

train_dataset = NucleiDataset(DATA_DIR, mask_df=mask_df, transform=train_transform, is_test=False)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

print(f"Loaded {len(train_dataset)} training images")
print(f"Batch size: {BATCH_SIZE} | Num workers: {NUM_WORKERS} | Image size: {IMAGE_HEIGHT}x{IMAGE_WIDTH}")
print()


Loaded 670 training images
Batch size: 8 | Num workers: 0 | Image size: 256x256



In [7]:
# ----------------------------
# Model, loss, optimizer
# ----------------------------
print("Initializing model...")
model = smp.Unet(encoder_name=ENCODER, encoder_weights=ENCODER_WEIGHTS, in_channels=3, classes=1, activation=None).to(DEVICE)

# parameter count 
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model: UNet with encoder={ENCODER} ({ENCODER_WEIGHTS})")
print(f"Trainable parameters: {count_parameters(model):,}")
print(f"Initial learning rate: {LR}")
print()

bce = nn.BCEWithLogitsLoss()

def dice_loss(preds, targets, eps=1e-7):
    preds = torch.sigmoid(preds)
    preds = preds.view(-1)
    targets = targets.view(-1)
    inter = (preds * targets).sum()
    union = preds.sum() + targets.sum()
    dice = (2. * inter + eps) / (union + eps)
    return 1 - dice


def loss_fn(preds, targets):
    return 0.5 * bce(preds, targets) + 0.5 * dice_loss(preds, targets)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)




Initializing model...
Model: UNet with encoder=efficientnet-b3 (imagenet)
Trainable parameters: 13,159,033
Initial learning rate: 0.0001



In [8]:
# ----------------------------
# Visualization helpers 
# ----------------------------
def unnormalize_image(tensor_image):
    """tensor_image: C x H x W (torch tensor or numpy) in normalized [0,1] with ImageNet mean/std applied"""
    if isinstance(tensor_image, torch.Tensor):
        img = tensor_image.detach().cpu().numpy()
    else:
        img = tensor_image.copy()
    # if shape CxHxW
    if img.shape[0] == 3:
        img = np.transpose(img, (1,2,0))
    # unnormalize
    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])
    img = (img * std) + mean
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)
    return img  # H x W x 3 (RGB)


def resize_and_pad(img, target_w, target_h, pad_value=0):
    """Resize `img` keeping aspect ratio then pad to (target_h, target_w). img is HxWxC or HxW."""
    if img is None:
        return None
    h, w = img.shape[:2]
    if h == 0 or w == 0:
        return None
    # compute scale to fit into target box
    scale = min(target_w / float(w), target_h / float(h))
    new_w = max(1, int(w * scale))
    new_h = max(1, int(h * scale))
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    # pad centered
    pad_left = (target_w - new_w) // 2
    pad_right = target_w - new_w - pad_left
    pad_top = (target_h - new_h) // 2
    pad_bottom = target_h - new_h - pad_top
    if len(resized.shape) == 2:
        # grayscale
        padded = cv2.copyMakeBorder(resized, pad_top, pad_bottom, pad_left, pad_right,
                                    borderType=cv2.BORDER_CONSTANT, value=pad_value)
    else:
        padded = cv2.copyMakeBorder(resized, pad_top, pad_bottom, pad_left, pad_right,
                                    borderType=cv2.BORDER_CONSTANT, value=[pad_value, pad_value, pad_value])
    return padded


def save_visual_grid(epoch, model, dataset, sample_ids, out_path, device=DEVICE, threshold=THRESHOLD):
    model.eval()
    DISPLAY_W, DISPLAY_H = IMAGE_WIDTH, IMAGE_HEIGHT
    rows = []
    with torch.no_grad():
        for sid in sample_ids:
            img_glob = glob(os.path.join(dataset.base_dir, sid, 'images', '*'))
            if not img_glob:
                continue
            orig = cv2.imread(img_glob[0], cv2.IMREAD_COLOR)
            if orig is None:
                continue
            orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
            h, w = orig.shape[:2]

            # prepare input for model
            aug = valid_transform(image=orig, mask=np.zeros((h, w), dtype=np.uint8))
            inp = aug['image'].unsqueeze(0).to(device)

            preds = model(inp)
            probs = torch.sigmoid(preds).squeeze().detach().cpu().numpy()
            probs_resized = cv2.resize(probs, (w, h), interpolation=cv2.INTER_LINEAR)
            mask_pred = (probs_resized >= threshold).astype(np.uint8) * 255

            # GT mask
            records = dataset.mask_df[dataset.mask_df['ImageId'] == sid]
            gt_mask = np.zeros((h, w), dtype=np.uint8)
            for _, r in records.iterrows():
                gt_mask += rle_decode(r['EncodedPixels'], (h, w)).astype(np.uint8)
            gt_mask = np.clip(gt_mask, 0, 1) * 255

            inp_img = orig.copy()
            gt_colored = inp_img.copy(); gt_colored[gt_mask == 255] = [255, 0, 0]
            pred_colored = inp_img.copy(); pred_colored[mask_pred == 255] = [0, 255, 0]

            # resize + pad each tile to DISPLAY_W x DISPLAY_H
            inp_tile  = resize_and_pad(inp_img, DISPLAY_W, DISPLAY_H, pad_value=0)
            gt_tile   = resize_and_pad(gt_colored, DISPLAY_W, DISPLAY_H, pad_value=0)
            pred_tile = resize_and_pad(pred_colored, DISPLAY_W, DISPLAY_H, pad_value=0)

            if inp_tile is None or gt_tile is None or pred_tile is None:
                continue

            # annotate
            def annotate(img, text):
                canvas = img.copy()
                cv2.putText(canvas, text, (8, 18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
                return canvas

            inp_annot  = annotate(inp_tile,  f"ID:{sid} - Input")
            gt_annot   = annotate(gt_tile,   "GT mask (red)")
            pred_annot = annotate(pred_tile, "Pred mask (green)")

            row = np.hstack([inp_annot, gt_annot, pred_annot])
            rows.append(row)

    if len(rows) == 0:
        print("No samples found to visualize.")
        return

    grid = np.vstack(rows)
    grid_bgr = cv2.cvtColor(grid, cv2.COLOR_RGB2BGR)
    cv2.imwrite(out_path, grid_bgr)
    print(f"Saved visualization for epoch {epoch} to: {out_path}")





In [ ]:
# ----------------------------
# Training loop
# ----------------------------
print('\n--- Starting Training ---\n')
start_time_all = time.time()

# storage for metrics to plot after training
epoch_losses = []
epoch_dices = []

for epoch in range(1, EPOCHS+1):
    epoch_start = time.time()
    model.train()
    epoch_loss = 0.0
    batches = len(train_loader)
    print(f"Epoch {epoch}/{EPOCHS} - starting. LR={optimizer.param_groups[0]['lr']} - batches={batches}")
    batch_idx = 0

    # batch-level dice accumulator for this epoch
    epoch_batch_dice_sum = 0.0
    epoch_batch_count = 0

    # A short progress bar for batches 
    for images, masks, ids in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False):
        batch_idx += 1
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)

        optimizer.zero_grad()
        preds = model(images)
        loss = loss_fn(preds, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # compute batch dice score (thresholded at 0.5 on sigmoid)
        with torch.no_grad():
            probs = torch.sigmoid(preds)
            probs_bin = (probs >= THRESHOLD).float()
            # compute dice for batch
            # flatten both
            p_flat = probs_bin.view(-1)
            t_flat = masks.view(-1)
            inter = (p_flat * t_flat).sum()
            union = p_flat.sum() + t_flat.sum()
            dice_batch = ((2. * inter + 1e-7) / (union + 1e-7)).item()
            epoch_batch_dice_sum += dice_batch
            epoch_batch_count += 1

        # print an occasional batch-level line for more visibility (every 50 batches or last batch)
        if batch_idx % 50 == 0 or batch_idx == batches:
            print(f"  [Epoch {epoch}] Batch {batch_idx}/{batches} - batch loss: {loss.item():.6f} - batch dice: {dice_batch:.4f}")

    avg_loss = epoch_loss / max(1, len(train_loader))
    # average dice for epoch
    avg_epoch_dice = epoch_batch_dice_sum / max(1, epoch_batch_count)

    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start
    print(f"Epoch {epoch} completed - Avg Loss: {avg_loss:.6f} - Avg Dice: {avg_epoch_dice:.6f} - Time: {epoch_time:.2f}s")

    # save metrics
    epoch_losses.append(avg_loss)
    epoch_dices.append(avg_epoch_dice)

    # checkpointing
    if epoch % CHECKPOINT_EVERY == 0:
        cp = f"checkpoint_epoch{epoch}.pth"
        try:
            torch.save(model.state_dict(), cp)
            print(f"--> Checkpoint saved: {cp}")
        except Exception as e:
            print(f"Warning: could not save checkpoint {cp} - {e}")

    # visualization: save a grid of VIS_SAMPLES examples every VIS_EPOCH_FREQ epochs
    if epoch % VIS_EPOCH_FREQ == 0:
        sample_ids = []
        sample_ids = train_dataset.ids[:VIS_SAMPLES]
        vis_out = os.path.join(VIS_DIR, f"epoch_{epoch}_visual.png")
        try:
            save_visual_grid(epoch, model, train_dataset, sample_ids, vis_out, device=DEVICE)
        except Exception as e:
            print(f"Failed to save visuals at epoch {epoch}: {e}")

print("\n--- Training Complete ---")
end_time_all = time.time()
total_minutes = (end_time_all - start_time_all) / 60.0
print(f"Total training time: {total_minutes:.2f} minutes.")
# Final save
try:
    torch.save(model.state_dict(), MODEL_SAVE)
    print("Final model saved as:", MODEL_SAVE)
except Exception as e:
    print(f"Warning: final model could not be saved to {MODEL_SAVE} - {e}")

# --- Plotting after training
try:
    import matplotlib
    matplotlib.use('Agg')  # safe backend for scripts (no display)
    import matplotlib.pyplot as plt

    plots_dir = os.path.join(BASE_DIR, 'training_plots')
    os.makedirs(plots_dir, exist_ok=True)

    epochs_range = list(range(1, len(epoch_losses) + 1))

    # Loss plot
    plt.figure()
    plt.plot(epochs_range, epoch_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Avg Training Loss')
    plt.title('Training Loss per Epoch')
    plt.grid(True)
    loss_plot_path = os.path.join(plots_dir, 'loss_vs_epoch.png')
    plt.savefig(loss_plot_path, bbox_inches='tight')
    plt.close()
    print(f"Saved loss plot to: {loss_plot_path}")

    # Dice plot
    plt.figure()
    plt.plot(epochs_range, epoch_dices)
    plt.xlabel('Epoch')
    plt.ylabel('Avg Training Dice')
    plt.title('Training Dice (accuracy) per Epoch')
    plt.grid(True)
    dice_plot_path = os.path.join(plots_dir, 'dice_vs_epoch.png')
    plt.savefig(dice_plot_path, bbox_inches='tight')
    plt.close()
    print(f"Saved dice plot to: {dice_plot_path}")

    # Combined plot (loss and dice) - secondary y-axis
    plt.figure()
    fig, ax1 = plt.subplots()
    ax1.plot(epochs_range, epoch_losses, label='Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax2 = ax1.twinx()
    ax2.plot(epochs_range, epoch_dices, label='Dice', linestyle='--')
    ax2.set_ylabel('Dice')
    plt.title('Training Loss and Dice per Epoch')
    # Adding legends
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right')
    combined_plot_path = os.path.join(plots_dir, 'loss_and_dice.png')
    fig.savefig(combined_plot_path, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved combined plot to: {combined_plot_path}")

    # Also save numpy arrays to disk for later plotting/analysis
    np.save(os.path.join(plots_dir, 'epoch_losses.npy'), np.array(epoch_losses))
    np.save(os.path.join(plots_dir, 'epoch_dices.npy'), np.array(epoch_dices))
    print(f"Saved metric arrays to: {plots_dir}")

except Exception as e:
    print(f"Failed to create/save plots: {e}")




--- Starting Training ---

Epoch 1/80 - starting. LR=0.0001 - batches=84


Epoch 1/80:  60%|█████▉    | 50/84 [00:41<00:22,  1.51it/s]

  [Epoch 1] Batch 50/84 - batch loss: 0.583054 - batch dice: 0.5901


  [Epoch 1] Batch 84/84 - batch loss: 0.502407 - batch dice: 0.6910
Epoch 1 completed - Avg Loss: 0.691532 - Avg Dice: 0.425955 - Time: 69.48s
Epoch 2/80 - starting. LR=0.0001 - batches=84


Epoch 2/80:  60%|█████▉    | 50/84 [00:40<00:35,  1.04s/it]

...

  [Epoch 79] Batch 84/84 - batch loss: 0.037334 - batch dice: 0.9597
Epoch 79 completed - Avg Loss: 0.072451 - Avg Dice: 0.925535 - Time: 59.22s
Epoch 80/80 - starting. LR=0.0001 - batches=84


Epoch 80/80:  60%|█████▉    | 50/84 [00:34<00:22,  1.49it/s]

  [Epoch 80] Batch 50/84 - batch loss: 0.053875 - batch dice: 0.9451


  [Epoch 80] Batch 84/84 - batch loss: 0.035809 - batch dice: 0.9670
Epoch 80 completed - Avg Loss: 0.067184 - Avg Dice: 0.931368 - Time: 56.16s
--> Checkpoint saved: checkpoint_epoch80.pth


Saved visualization for epoch 80 to: c:\Users\chama\Desktop\Project\training_visuals\epoch_80_visual.png

--- Training Complete ---
Total training time: 168.72 minutes.
Final model saved as: c:\Users\chama\Desktop\Project\unet_effnet_nuclei.pth
Saved loss plot to: c:\Users\chama\Desktop\Project\training_plots\loss_vs_epoch.png
Saved dice plot to: c:\Users\chama\Desktop\Project\training_plots\dice_vs_epoch.png
Saved combined plot to: c:\Users\chama\Desktop\Project\training_plots\loss_and_dice.png
Saved metric arrays to: c:\Users\chama\Desktop\Project\training_plots


In [10]:
# ----------------------------
# Inference helper 
# ----------------------------
def run_inference_on_dir(model, data_dir, output_csv_path, threshold=THRESHOLD):
    if not os.path.exists(data_dir):
        print(f"Data dir {data_dir} not found. Skipping inference for {data_dir}.")
        return

    ids = [os.path.basename(p) for p in glob(os.path.join(data_dir, '*'))]
    ds = NucleiDataset(data_dir, image_ids=ids, transform=valid_transform, is_test=True)
    dl = DataLoader(ds, batch_size=1, shuffle=False, num_workers=NUM_WORKERS)

    model.eval()
    rows = []
    with torch.no_grad():
        for img, _, img_id in tqdm(dl, desc=f'Predicting ({os.path.basename(data_dir)})'):
            img = img.to(DEVICE)
            preds = model(img)
            probs = torch.sigmoid(preds)
            probs = probs.squeeze(0).squeeze(0).cpu().numpy()

            # get original size using the image path
            img_glob = glob(os.path.join(data_dir, img_id[0], 'images', '*'))
            if img_glob:
                orig = cv2.imread(img_glob[0], cv2.IMREAD_COLOR)
                if orig is not None:
                    h, w = orig.shape[:2]
                    probs_resized = cv2.resize(probs, (w, h), interpolation=cv2.INTER_LINEAR)
                else:
                    probs_resized = probs
                    h, w = probs_resized.shape
            else:
                probs_resized = probs
                h, w = probs_resized.shape

            mask_bin = (probs_resized >= threshold).astype(np.uint8)

            if mask_bin.sum() == 0:
                encoded = ''
            else:
                encoded = rle_encode(mask_bin)

            rows.append({'ImageId': img_id[0], 'EncodedPixels': encoded})

    out_df = pd.DataFrame(rows)
    out_df.to_csv(output_csv_path, index=False)
    print(f"Inference complete for {data_dir}. Saved to {output_csv_path}")



In [11]:
# ----------------------------
# Run inference on Test 
# ----------------------------
print("\n--- Running Inference on Test Set ---\n")
if os.path.exists(TEST_DATA_DIR):
    run_inference_on_dir(model, TEST_DATA_DIR, SUBMISSION_PATH)
else:
    print(f"Test data dir {TEST_DATA_DIR} not found. Skipping Test inference.")




--- Running Inference on Test Set ---



Predicting (Test): 100%|██████████| 65/65 [00:07<00:00,  9.19it/s]


Inference complete for c:\Users\chama\Desktop\Project\data\Test. Saved to c:\Users\chama\Desktop\Project\submission.csv


In [ ]:
# ----------------------------
#  Run inference on Test2 
# ----------------------------
print("\n--- Running Inference on Test2 Set (if present) ---\n")
run_inference_on_dir(model, TEST2_DATA_DIR, TEST2_SUBMISSION_PATH)



--- Running Inference on Test2 Set (if present) ---



Predicting (Test2): 100%|██████████| 3019/3019 [04:03<00:00, 12.38it/s]


Inference complete for c:\Users\chama\Desktop\Project\data\Test2. Saved to c:\Users\chama\Desktop\Project\submission_test2.csv


In [13]:
# ----------------------------
# Evaluation: compare submission.csv to ground truth
# ----------------------------
print('\n--- Evaluating Submission vs Ground Truth ---\n')

if not os.path.exists(GROUND_TRUTH_PATH):
    print(f"Ground truth file not found at {GROUND_TRUTH_PATH}. Cannot evaluate.")
else:
    gt_df = pd.read_csv(GROUND_TRUTH_PATH)
    pred_df = pd.read_csv(SUBMISSION_PATH)
    image_ids = np.unique(pd.concat([gt_df['ImageId'], pred_df['ImageId']]).dropna().unique())

    bce = nn.BCEWithLogitsLoss()

    def dice_score_pytorch(preds, targets, eps=1e-7):
        preds = preds.float().reshape(-1)
        targets = targets.float().reshape(-1)
        inter = (preds * targets).sum()
        union = preds.sum() + targets.sum()
        dice = (2. * inter + eps) / (union + eps)
        return dice

    def dice_loss_pytorch(preds, targets, eps=1e-7):
        return 1.0 - dice_score_pytorch(preds, targets, eps)

    def hybrid_loss_pytorch(preds, targets):
        bce_term = bce(preds, targets)
        dice_term = dice_loss_pytorch(preds, targets)
        return 0.5 * bce_term + 0.5 * dice_term

    # new: IoU (Jaccard) score for binary masks
    def iou_score_pytorch(preds, targets, eps=1e-7):
        """
        preds, targets: torch tensors with values 0/1 (or floats interpreted as probabilities but here we pass binary masks)
        returns IoU scalar tensor
        """
        preds = preds.float().reshape(-1)
        targets = targets.float().reshape(-1)
        inter = (preds * targets).sum()
        union = preds.sum() + targets.sum() - inter
        iou = (inter + eps) / (union + eps)
        return iou

    total_dice = 0.0
    total_loss = 0.0
    total_iou = 0.0
    evaluated_count = 0

    for image_id in tqdm(image_ids, desc='Calculating Metrics'):
        # find test image for dims
        image_glob = glob(os.path.join(TEST_DATA_DIR, image_id, 'images', '*'))
        if not image_glob:
            continue
        orig = cv2.imread(image_glob[0], cv2.IMREAD_COLOR)
        if orig is None:
            continue
        h, w = orig.shape[:2]
        shape = (h, w)

        # GT mask (aggregate RLEs)
        gt_records = gt_df[gt_df['ImageId'] == image_id]
        gt_mask_np = np.zeros(shape, dtype=np.uint8)
        for _, row in gt_records.iterrows():
            gt_mask_np += rle_decode(row['EncodedPixels'], shape).astype(np.uint8)
        gt_mask_np = np.clip(gt_mask_np, 0, 1)

        # Pred mask
        pred_records = pred_df[pred_df['ImageId'] == image_id]
        pred_mask_np = np.zeros(shape, dtype=np.uint8)
        for _, row in pred_records.iterrows():
            if pd.isna(row['EncodedPixels']) or row['EncodedPixels'] == '':
                continue
            pred_mask_np += rle_decode(row['EncodedPixels'], shape).astype(np.uint8)
        pred_mask_np = np.clip(pred_mask_np, 0, 1)

        gt_mask_tensor = torch.from_numpy(gt_mask_np).unsqueeze(0).unsqueeze(0).to(DEVICE).float()
        pred_mask_tensor = torch.from_numpy(pred_mask_np).unsqueeze(0).unsqueeze(0).to(DEVICE).float()

        dice = dice_score_pytorch(pred_mask_tensor, gt_mask_tensor)
        total_dice += dice.item()

        loss = hybrid_loss_pytorch(pred_mask_tensor, gt_mask_tensor)
        total_loss += loss.item()

        iou = iou_score_pytorch(pred_mask_tensor, gt_mask_tensor)
        total_iou += iou.item()

        evaluated_count += 1


    print()
    if evaluated_count == 0:
        print('No images were evaluated (check that test images and ground truth RLEs are aligned).')
    else:
        avg_dice = total_dice / evaluated_count
        avg_loss = total_loss / evaluated_count
        avg_iou = total_iou / evaluated_count
        print('\n' + '='*60)
        print('    MODEL ACCURACY ON TEST DATA (GROUND TRUTH) ')
        print('='*60)
        print(f'Images evaluated      : {evaluated_count}')
        print(f'Average Dice Score    : {avg_dice:.6f}')
        print(f'Average IoU (Jaccard) : {avg_iou:.6f}')
        print(f'Average Hybrid Loss   : {avg_loss:.6f}')
        print('='*60)



--- Evaluating Submission vs Ground Truth ---



Calculating Metrics: 100%|██████████| 65/65 [00:03<00:00, 17.76it/s]



    MODEL ACCURACY ON TEST DATA (GROUND TRUTH) 
Images evaluated      : 65
Average Dice Score    : 0.825473
Average IoU (Jaccard) : 0.723451
Average Hybrid Loss   : 0.415633
